In [1]:
from utils import *
from detectors import torch_script_module
from carla_control import carla_client
import numpy as np
from mayavi import mlab
import torch
import time

logger = create_logger()
CFG_FILE = ".\\configs\\observe_predict.yaml"
cfg = cfg_from_yaml_file(CFG_FILE, cfg)
print(cfg)

pcs_dataset = file_point_cloud_dataset(dataset_cfg = cfg.DATA_CONFIG, logger=logger, root_path=".\\dataset", class_names=cfg.CLASS_NAMES)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
{'ROOT_DIR': WindowsPath('D:/Python_project/beamng_python_experiment'), 'LOCAL_RANK': 0, 'CLASS_NAMES': ['Car', 'Pedestrian', 'Cyclist'], 'DATA_CONFIG': {'POINT_CLOUD_RANGE': [0, -39.68, -3, 69.12, 39.68, 1], 'POINT_FEATURE_ENCODING': {'encoding_type': 'absolute_coordinates_encoding', 'used_feature_list': ['x', 'y', 'z', 'intensity'], 'src_feature_list': ['x', 'y', 'z', 'intensity']}, 'PREVIEW_CHANNEL': ['detector'], 'DATA_PROCESSOR': {'detector': [{'NAME': 'mask_points_by_range', 'POINT_CLOUD_RANGE': [0, -39.68, -3, 69.12, 39.68, 1]}, {'NAME': 'transform_points_to_voxels', 'VOXEL_SIZE': [0.16, 0.16, 4], 'MAX_POINTS_PER_VOXEL': 32, 'MAX_NUMBER_OF_VOXELS': {'train': 16000, 'test': 40000}}]}}, 'MODEL': {'NAME': 'PointPillar', 'POST_PROCESSING': {'RECALL_THRESH_LIST': [0.3, 0.5, 0.7], 'SCORE_THRESH': 0.1, 'OUTPUT_RAW_SCORE

In [2]:
model = torch.jit.load("./torch_scripts/point_pillar_model.pt")
model.cuda()
model.eval()

def model_forwarding(idx, data_dict):
    final_boxes, final_scores, final_labels = None, None, None
    before_time = time.perf_counter()
    
    if data_dict["points"] is not None:
        pred_dicts, _ = model.forward(data_dict)
        
        cls_preds = pred_dicts["pred_scores"]
        box_preds = pred_dicts["pred_boxes"]
        label_preds = pred_dicts["pred_labels"]

        selected, selected_scores = class_agnostic_nms(
                            box_scores=cls_preds, box_preds=box_preds,
                            score_thresh=0.5
                        )

        final_scores = selected_scores
        final_labels = label_preds[selected]
        final_boxes = box_preds[selected]
    after_time = time.perf_counter()
    return after_time - before_time, data_dict, final_boxes, final_scores, final_labels

In [3]:

from utils import *
import mayavi.mlab as mlab


with torch.no_grad():
    data = pcs_dataset[1]
    
    for channel_name, channel in data.items():
        data_dict = data[channel_name]
        
        points = data_dict["points"]
        gt = data_dict["gt_boxes"]
        
        final_boxes, final_scores, final_labels = None, None, None
        
        data_dict = pcs_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)
    
    
        forward_time, data_dict, final_boxes, final_scores, final_labels = model_forwarding(0, data_dict)

        
        draw_scenes_maya(None,
                    points=points, ref_boxes=final_boxes,
                    ref_scores=final_scores, ref_labels=final_labels, gt_boxes=gt
                )
        
        mlab.show(stop=True)
        
        


In [ ]:
# import open3d as o3d
# import threading
# from utils import *

# def fun(vis:o3d.cpu.pybind.visualization.Visualizer):
#     print("run")
#     vis.run()
#     # vis.destroy_window()
    
# with torch.no_grad():
#     data_dict = pcs_dataset[0]
#     # print(data_dict)
#     for channel_name, channel in data_dict.items():
#         data = data_dict[channel_name]
#         # print(data_dict)
#         vis = o3d.visualization.Visualizer()
#         vis.create_window(window_name=channel_name)
#         points = data["points"]
#         draw_scenes(vis,
#                     points=points, ref_boxes=None,
#                     ref_scores=None, ref_labels=None, confidence=None
#                 )
        
#         thread = threading.Thread(target=lambda: fun(vis))
#         thread.start()
        

#     # ori_pc = data_dict["raw_points"]
#     # tran_pc = data_dict["points"]
#     # print(ori_pc - tran_pc)
#     # print(data_dict["points"].shape)
#     # print(data_dict["intensity"].sum())
#     # mlab.points3d(ori_pc[:, 0], ori_pc[:, 1], ori_pc[:, 2], data_dict["intensity"], mode='point', scale_factor=5)
#     # mlab.points3d(tran_pc[:, 0], tran_pc[:, 1], tran_pc[:, 2], tran_pc[:, 3], mode='point', scale_factor=5)

#     # # mlab.points3d(kitti_01_pcs[:, 0], kitti_01_pcs[:, 1], kitti_01_pcs[:, 2], kitti_01_pcs[:, 3], mode='point', scale_factor=5)
#     # mlab.show(stop=False)
#     # data_dict = pcs_dataset.collate_batch([data_dict])
#     # print(data_dict["points"].shape)
#     # load_data_to_gpu(data_dict)
#     # print(data_dict["points"].size())